# Запросы к БД с помощью PySpark

In [ ]:
# версия Spark
from pyspark.sql import functions as f
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
conf = SparkConf()
conf.setMaster("local").setAppName('My app')
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)
print('Запущен Spark версии', spark.version)

Запущен Spark версии 3.5.4


## Загрузим таблицы из БД BikeStore

In [9]:
products = spark.read.csv(
    "./data/products.csv",
    sep=",",
    inferSchema=True,
    header="true",
)

brands = spark.read.csv(
    "./data/brands.csv",
    sep=",",
    inferSchema=True,
    header="true",
)

categories = spark.read.csv(
    "./data/categories.csv",
    sep=",",
    inferSchema=True,
    header="true",
)

customers = spark.read.csv(
    "./data/customers.csv",
    sep=",",
    inferSchema=True,
    header="true",
)

order_items = spark.read.csv(
    "./data/order_items.csv",
    sep=",",
    inferSchema=True,
    header="true",
)

orders = spark.read.csv(
    "./data/orders.csv",
    sep=",",
    inferSchema=True,
    header="true",
)

staffs = spark.read.csv(
    "./data/staffs.csv",
    sep=",",
    inferSchema=True,
    header="true",
)

stocks = spark.read.csv(
    "./data/stocks.csv",
    sep=",",
    inferSchema=True,
    header="true",
)

stores = spark.read.csv(
    "./data/stores.csv",
    sep=",",
    inferSchema=True,
    header="true",
)

In [11]:
products.show(5)
brands.show(5)
categories.show(5)
customers.show(5)
order_items.show(5)
orders.show(5)
staffs.show(5)
stocks.show(5)
stores.show(5)

+----------+--------------------+--------+-----------+----------+----------+
|product_id|        product_name|brand_id|category_id|model_year|list_price|
+----------+--------------------+--------+-----------+----------+----------+
|         1|     Trek 820 - 2016|       9|          6|      2016|    379.99|
|         2|Ritchey Timberwol...|       5|          6|      2016|    749.99|
|         3|Surly Wednesday F...|       8|          6|      2016|    999.99|
|         4|Trek Fuel EX 8 29...|       9|          6|      2016|   2899.99|
|         5|Heller Shagamaw F...|       3|          6|      2016|   1320.99|
+----------+--------------------+--------+-----------+----------+----------+
only showing top 5 rows

+--------+-----------+
|brand_id| brand_name|
+--------+-----------+
|       1|    Electra|
|       2|       Haro|
|       3|     Heller|
|       4|Pure Cycles|
|       5|    Ritchey|
+--------+-----------+
only showing top 5 rows

+-----------+-------------------+
|category_id|   

### Задача 1

Напишите запрос, чтобы получить все названия продуктов и соответствующие им торговые марки (brand).

In [12]:
result_1 = products.join(brands, products["brand_id"] == brands["brand_id"], "inner")
result_1 = result_1.select("product_name", "brand_name")
result_1.show()

+--------------------+-----------+
|        product_name| brand_name|
+--------------------+-----------+
|     Trek 820 - 2016|       Trek|
|Ritchey Timberwol...|    Ritchey|
|Surly Wednesday F...|      Surly|
|Trek Fuel EX 8 29...|       Trek|
|Heller Shagamaw F...|     Heller|
|Surly Ice Cream T...|      Surly|
|Trek Slash 8 27.5...|       Trek|
|Trek Remedy 29 Ca...|       Trek|
|Trek Conduit+ - 2016|       Trek|
|Surly Straggler -...|      Surly|
|Surly Straggler 6...|      Surly|
|Electra Townie Or...|    Electra|
|Electra Cruiser 1...|    Electra|
|Electra Girl's Ha...|    Electra|
|Electra Moto 1 - ...|    Electra|
|Electra Townie Or...|    Electra|
|Pure Cycles Vine ...|Pure Cycles|
|Pure Cycles Weste...|Pure Cycles|
|Pure Cycles Willi...|Pure Cycles|
|Electra Townie Or...|    Electra|
+--------------------+-----------+
only showing top 20 rows



### Задача 2

Напишите запрос, чтобы найти всех активных сотрудников и наименования магазинов, в которых они работают.

In [13]:
result_2 = staffs.join(stores, staffs["store_id"] == stores["store_id"], "inner")
result_2 = result_2.select("first_name", "last_name", "store_name")
result_2.show()

+----------+---------+----------------+
|first_name|last_name|      store_name|
+----------+---------+----------------+
|   Fabiola|  Jackson|Santa Cruz Bikes|
|    Mireya| Copeland|Santa Cruz Bikes|
|     Genna|  Serrano|Santa Cruz Bikes|
|    Virgie|  Wiggins|Santa Cruz Bikes|
|  Jannette|    David|   Baldwin Bikes|
| Marcelene|    Boyer|   Baldwin Bikes|
|    Venita|   Daniel|   Baldwin Bikes|
|      Kali|   Vargas|   Rowlett Bikes|
|     Layla|  Terrell|   Rowlett Bikes|
|Bernardine|  Houston|   Rowlett Bikes|
+----------+---------+----------------+



### Задача 3

Напишите запрос, чтобы перечислить всех покупателей выбранного магазина с указанием их полных имен, электронной почты и номера телефона.

In [23]:
join_c_o = customers.join(orders, customers["customer_id"] == orders["customer_id"], "inner")

result_3 = join_c_o.join(stores, join_c_o["store_id"] == stores["store_id"], "inner")

result_3 = result_3.filter(result_3["store_name"] == "Santa Cruz Bikes")
result_3 = result_3.select(
    join_c_o["first_name"],
    join_c_o["last_name"],
    join_c_o["email"],
    join_c_o["phone"],
).distinct()
result_3.show()

+----------+---------+--------------------+--------------+
|first_name|last_name|               email|         phone|
+----------+---------+--------------------+--------------+
|   Pearlie|  Cochran|pearlie.cochran@m...|          NULL|
|    Desire|  Mcgowan|desire.mcgowan@ms...|(661) 800-8091|
|    Jeffry|   Church|jeffry.church@msn...|          NULL|
|     Shara|     Pope|  shara.pope@msn.com|          NULL|
|   Novella|     Ross|novella.ross@gmai...|          NULL|
|   Arvilla|   Osborn|arvilla.osborn@gm...|          NULL|
|  Mercedez|   Brooks|mercedez.brooks@h...|          NULL|
|        Ji|     Burt| ji.burt@hotmail.com|          NULL|
|    Elvera|     Peck|elvera.peck@yahoo...|          NULL|
|      Adam|Henderson|adam.henderson@ho...|          NULL|
|    Willow|  Gardner|willow.gardner@gm...|          NULL|
|       Ara|  Vazquez| ara.vazquez@msn.com|          NULL|
|  Graciela|   Barber|graciela.barber@h...|(805) 656-6987|
|  Giovanna|Jefferson|giovanna.jefferso...|          NUL

### Задача 4

Напишите запрос для подсчета количества продуктов в каждой категории.

In [24]:
result_4 = products.join(categories, products["category_id"] == categories["category_id"], "inner")
result_4 = result_4.groupBy("category_name").count()
result_4.show()

+-------------------+-----+
|      category_name|count|
+-------------------+-----+
|     Electric Bikes|   24|
|     Mountain Bikes|   60|
|   Comfort Bicycles|   30|
|         Road Bikes|   60|
|Cyclocross Bicycles|   10|
|  Children Bicycles|   59|
|  Cruisers Bicycles|   78|
+-------------------+-----+



### Задача 5

Напишите запрос, чтобы указать общее количество заказов для каждого клиента.

In [31]:
from pyspark.sql.functions import desc

result_5 = customers.join(orders, customers["customer_id"] == orders["customer_id"], "inner")
result_5 = result_5.groupBy(
    customers["customer_id"], customers["first_name"], customers["last_name"]
).agg({"order_id": "count"})
result_5 = result_5.select("first_name", "last_name", "count(order_id)")
result_5 = result_5.orderBy(desc("count(order_id)"))
result_5.show()

+----------+---------+---------------+
|first_name|last_name|count(order_id)|
+----------+---------+---------------+
|  Lizzette|    Stein|              3|
|     Ronna|   Butler|              3|
|    Tameka|   Fisher|              3|
| Saturnina|   Garner|              3|
| Jacquline|   Duncan|              3|
|     Garry| Espinoza|              3|
|    Corene|     Wall|              3|
|    Lorrie|   Becker|              3|
|     Aleta|  Shepard|              3|
|     Jayne| Kirkland|              3|
|     Kasha|     Todd|              3|
|     Tobie|   Little|              3|
|   Merlene|   Vinson|              3|
|    Bobbie|   Foster|              3|
|    Linnie|   Branch|              3|
| Bridgette|   Guerra|              3|
|   Mozelle|   Carter|              3|
|   Pamelia|   Newman|              3|
|    Monika|     Berg|              3|
|   Lashawn|    Ortiz|              3|
+----------+---------+---------------+
only showing top 20 rows



### Задача 6

Напишите запрос, в котором будет указана информация о полном имени и общем количестве заказов клиентов, которые хотя бы 1 раз сделали заказ.

In [32]:
result_6 = customers.join(orders, customers["customer_id"] == orders["customer_id"], "inner")
result_6 = result_6.groupBy(
    customers["customer_id"], customers["first_name"], customers["last_name"]
).agg({"order_id": "count"})
result_6 = result_6.select("first_name", "last_name", "count(order_id)")
result_6 = result_6.filter(result_6["count(order_id)"] >= 1)
result_6 = result_6.orderBy("count(order_id)")
result_6.show()

+----------+---------+---------------+
|first_name|last_name|count(order_id)|
+----------+---------+---------------+
|     Sally|   Kinney|              1|
|  Caroline|  Jenkins|              1|
|     Glady|    Wells|              1|
|       Bee|    Baker|              1|
| Nathaniel|  Richard|              1|
|  Genoveva|    Tyler|              1|
|    Rudolf|  Gilliam|              1|
|   Suellen|  Mercado|              1|
|   Shantel|  Gregory|              1|
|     Julia|   Joyner|              1|
|   Stefany|   Potter|              1|
|   Elenore|  William|              1|
|  Rochelle|     Ward|              1|
|   Shemeka|    Lyons|              1|
|  Casimira|  Chapman|              1|
|      Aron|  Wiggins|              1|
|    Keitha|    Black|              1|
|   Rochell| Cantrell|              1|
|      Sung| Chambers|              1|
|    Edmund|   Gaines|              1|
+----------+---------+---------------+
only showing top 20 rows



### Задача 7

Напишите запрос для расчета общего объема продаж по каждому продукту (с учетом количества продукта, его цены по прейскуранту и скидки).

In [35]:
join_p_oi = products.join(order_items, products["product_id"] == order_items["product_id"], "inner")
result_7 = join_p_oi.groupBy("product_name").agg(
    (
        f.sum(join_p_oi["quantity"] * order_items["list_price"] * (1 - join_p_oi["discount"]))
    ).alias("total")
)
result_7 = result_7.withColumn("total", f.round(result_7["total"], 3))
result_7.show()

+--------------------+----------+
|        product_name|     total|
+--------------------+----------+
|Electra Straight ...|   362.691|
|Electra Heartchya...|   1929.54|
|Trek Precaliber 2...|   255.992|
|Electra Townie Co...|  1646.982|
|Electra Townie Ba...|  6497.928|
|Electra Tiger Sha...|  1804.744|
|Trek Stache Carbo...|  4149.155|
|Heller Bloodhound...|   8992.54|
|Trek Precaliber 2...|   640.083|
|Electra Townie Or...|  1574.965|
|Electra Townie Or...|  4963.927|
|Electra Amsterdam...|  3035.975|
|Electra Townie Or...| 29164.205|
|Trek Slash 8 27.5...|555558.611|
|Trek Boy's Kickst...|  3236.784|
|Electra Townie Or...|  2069.954|
|Sun Bicycles Spid...| 19491.966|
|Trek Remedy 7 27....| 14069.953|
|Trek Domane ALR 4...|  9749.437|
|Trek Farley Carbo...|  7239.928|
+--------------------+----------+
only showing top 20 rows



### Задача 8

Напишите запрос с расчетом количества заказов по каждому статусу заказа.

In [38]:
result_8 = (orders.groupBy("order_status")
    .count()
    .orderBy("order_status")
)
result_8.show()

+------------+-----+
|order_status|count|
+------------+-----+
|           1|   62|
|           2|   63|
|           3|   45|
|           4| 1445|
+------------+-----+



### Задача 9

Напишите запрос для расчета общей суммы продаж за каждый месяц.

In [ ]:
join_o_oi = orders.join(order_items, orders["order_id"] == order_items["order_id"], "inner")

result_9 = join_o_oi.select(f.month("order_date").alias("month"), "quantity", "list_price", "discount")

result_9 = result_9.groupBy("month").agg(
    (f.sum(result_9["quantity"] * result_9["list_price"] * (1 - result_9["discount"]))).alias("total_sum_per_month")
)
result_9 = result_9.withColumn(
    "total_sum_per_month", f.round(result_9["total_sum_per_month"], 3)
)
result_9 = result_9.sort("month")
result_9.show()

+-----+-------------------+
|month|total_sum_per_month|
+-----+-------------------+
|    1|         882193.007|
|    2|         669694.131|
|    3|         853503.224|
|    4|        1212356.825|
|    5|         473503.247|
|    6|         589616.769|
|    7|         440890.107|
|    8|         524588.647|
|    9|         575460.831|
|   10|         526187.518|
|   11|         475269.321|
|   12|         465852.931|
+-----+-------------------+



### Задача 10

Напишите запрос, чтобы найти топ 5 клиентов, которые потратили больше всего денег.

In [42]:
join_o_c = orders.join(customers, orders["customer_id"] == customers["customer_id"], "inner")

result_10 = join_o_c.join(order_items, join_o_c["order_id"] == order_items["order_id"], "inner")

result_10 = result_10.groupBy(
    customers["customer_id"], customers["first_name"], customers["last_name"]
).agg(
    (f.sum(result_10["quantity"] * result_10["list_price"] * (1 - result_10["discount"]))).alias("total_sum")
)
result_10 = (
    result_10.sort("total_sum", ascending=False)
    .select("first_name", "last_name", "total_sum")
    .limit(5)
)
result_10 = result_10.withColumn("total_sum", f.round(result_10["total_sum"], 3))
result_10.show()

+----------+---------+---------+
|first_name|last_name|total_sum|
+----------+---------+---------+
|    Sharyn|  Hopkins|34807.939|
|   Pamelia|   Newman| 33634.26|
|      Abby|   Gamble|32803.006|
|   Lyndsey|     Bean|32675.072|
|    Emmitt|  Sanchez|31925.886|
+----------+---------+---------+

